# MMAI5040 - Business Application of AI 1

## Class 4.1 Regression Frameworks for Machine Learning: Multiple Linear Regressions

Import required packages

In [ ]:
!pip install dmba

In [ ]:
%matplotlib inline
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
import statsmodels.formula.api as sm
import matplotlib.pylab as plt

from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score


no display found. Using non-interactive Agg backend


## Load data and run linear regression model with scikits Linear Regression method

In [ ]:
# Reduce data frame to the top 1000 rows and select columns for regression analysis
car_df = pd.read_csv('ToyotaCorolla.csv')
car_df = car_df.iloc[0:1000]

# select outcome and predictor variables
predictors = ['Age_08_04', 'KM', 'Fuel_Type', 'HP', 'Met_Color', 'Automatic', 'CC', 'Doors', 'Quarterly_Tax', 'Weight']
outcome = 'Price'

# partition data
X = pd.get_dummies(car_df[predictors], drop_first=True) # creates (n-1) dummies for categorical predictor Fuel-type
y = car_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

# fit a linear regression model
car_lm = LinearRegression()
car_lm.fit(train_X, train_y)

# print coefficients
print('intercept ', car_lm.intercept_)
print(pd.DataFrame({'Predictor': X.columns, 'coefficient': car_lm.coef_}))

# print performance measures
regressionSummary(train_y, car_lm.predict(train_X))

intercept  -1319.3543800412208
           Predictor  coefficient
0          Age_08_04  -140.748761
1                 KM    -0.017840
2                 HP    36.103419
3          Met_Color    84.281830
4          Automatic   416.781954
5                 CC     0.017737
6              Doors   -50.657863
7      Quarterly_Tax    13.625325
8             Weight    13.038711
9   Fuel_Type_Diesel  1066.464681
10  Fuel_Type_Petrol  2310.249543

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 1400.5823
            Mean Absolute Error (MAE) : 1046.9072
          Mean Percentage Error (MPE) : -1.0223
Mean Absolute Percentage Error (MAPE) : 9.2994


In [ ]:
pred_y = car_lm.predict(train_X)

print('adjusted r2 : ', adjusted_r2_score(train_y, pred_y, car_lm))
print('AIC : ', AIC_score(train_y, pred_y, car_lm))
print('BIC : ', BIC_score(train_y, pred_y, car_lm))

adjusted r2 :  0.8537958550253093
AIC :  10422.298278332171
BIC :  10479.45836384998


In [ ]:
# Use predict() to make predictions on a new set
car_lm_pred = car_lm.predict(valid_X)

result = pd.DataFrame({'Predicted': car_lm_pred, 'Actual': valid_y,
                       'Residual': valid_y - car_lm_pred})
print(result.head(20))

# Compute common accuracy measures
regressionSummary(valid_y, car_lm_pred)

        Predicted  Actual     Residual
507  10607.333940   11500   892.666060
818   9272.705792    8950  -322.705792
452  10617.947808   11450   832.052192
368  13600.396275   11450 -2150.396275
242  12396.694660   11950  -446.694660
929   9496.498212    9995   498.501788
262  12480.063217   13500  1019.936783
810   8834.146068    7950  -884.146068
318  12183.361282    9900 -2283.361282
49   19206.965683   21950  2743.034317
446  10987.498309   11950   962.501691
142  18501.527375   19950  1448.472625
968   9914.690947    9950    35.309053
345  13827.299932   14950  1122.700068
971   7966.732543   10495  2528.267457
133  17185.242041   15950 -1235.242041
104  19952.658062   19450  -502.658062
6    16570.609280   16900   329.390720
600  13739.409113   11250 -2489.409113
496  11267.513740   11750   482.486260

Regression statistics

                      Mean Error (ME) : 103.6803
       Root Mean Squared Error (RMSE) : 1312.8523
            Mean Absolute Error (MAE) : 1017.5972
        

In [ ]:
# Determine the residuals and create a histogram
car_lm_pred = car_lm.predict(valid_X)
all_residuals = valid_y - car_lm_pred

# Determine the percentage of datapoints with a residual in [-1406, 1406] = approx. 75\%
print(len(all_residuals[(all_residuals > -1406) & (all_residuals < 1406)]) / len(all_residuals))

ax = pd.DataFrame({'Residuals': all_residuals}).hist(bins=25)

plt.tight_layout()
plt.show()

0.7425


## Selecting Predictor Subsets

### Exhaustive search

In [ ]:
# Run an exhaustive search. The Fuel type column is categorical and needs to be converted into dummy variables.

def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    pred_y = model.predict(train_X[variables])
    # we negate as score is optimized to be as low as possible
    return -adjusted_r2_score(train_y, pred_y, model)

allVariables = train_X.columns
results = exhaustive_search(allVariables, train_model, score_model)

data = []
for result in results:
    model = result['model']
    variables = result['variables']
    AIC = AIC_score(train_y, model.predict(train_X[variables]), model)
    
    d = {'n': result['n'], 'r2adj': -result['score'], 'AIC': AIC}
    d.update({var: var in result['variables'] for var in allVariables})
    data.append(d)
pd.set_option('display.width', 100)
print(pd.DataFrame(data, columns=('n', 'r2adj', 'AIC') + tuple(sorted(allVariables))))
pd.reset_option('display.width')

     n     r2adj           AIC  ...  Met_Color  Quarterly_Tax  Weight
0    1  0.767901  10689.712094  ...      False          False   False
1    2  0.801160  10597.910645  ...      False          False   False
2    3  0.829659  10506.084235  ...      False          False    True
3    4  0.846357  10445.174820  ...      False          False    True
4    5  0.849044  10435.578836  ...      False           True    True
5    6  0.853172  10419.932278  ...      False           True    True
6    7  0.853860  10418.104025  ...      False           True    True
7    8  0.854297  10417.290103  ...      False           True    True
8    9  0.854172  10418.789079  ...      False           True    True
9   10  0.854036  10420.330800  ...       True           True    True
10  11  0.853796  10422.298278  ...       True           True    True

[11 rows x 14 columns]


### Backward Elimination

In [ ]:
# Backward Elimination

def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = backward_elimination(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

Variables: Age_08_04, KM, HP, Met_Color, Automatic, CC, Doors, Quarterly_Tax, Weight, Fuel_Type_Diesel, Fuel_Type_Petrol
Start: score=10422.30
Step: score=10420.33, remove CC
Step: score=10418.79, remove Met_Color
Step: score=10417.29, remove Doors
Step: score=10417.29, remove None
['Age_08_04', 'KM', 'HP', 'Automatic', 'Quarterly_Tax', 'Weight', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol']


### Forward Selection

In [ ]:
# Forward Selection

# The initial model is the constant model - this requires special handling
# in train_model and score_model
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = forward_selection(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

Variables: Age_08_04, KM, HP, Met_Color, Automatic, CC, Doors, Quarterly_Tax, Weight, Fuel_Type_Diesel, Fuel_Type_Petrol
Start: score=11565.07, constant
Step: score=10689.71, add Age_08_04
Step: score=10597.91, add HP
Step: score=10506.08, add Weight
Step: score=10445.17, add KM
Step: score=10435.58, add Quarterly_Tax
Step: score=10419.93, add Fuel_Type_Petrol
Step: score=10418.10, add Fuel_Type_Diesel
Step: score=10417.29, add Automatic
Step: score=10417.29, add None
['Age_08_04', 'HP', 'Weight', 'KM', 'Quarterly_Tax', 'Fuel_Type_Petrol', 'Fuel_Type_Diesel', 'Automatic']


### Shrinkage Methods

In [ ]:
# Regularized (Shrinkage) Methods 

lasso = Lasso(normalize=True, alpha=1)
lasso.fit(train_X, train_y)
regressionSummary(valid_y, lasso.predict(valid_X))

lasso_cv = LassoCV(normalize=True, cv=5)
lasso_cv.fit(train_X, train_y)
regressionSummary(valid_y, lasso_cv.predict(valid_X))
print('Lasso-CV chosen regularization: ', lasso_cv.alpha_)
print(lasso_cv.coef_)

ridge = Ridge(normalize=True, alpha=1)
ridge.fit(train_X, train_y)
regressionSummary(valid_y, ridge.predict(valid_X))

bayesianRidge = BayesianRidge(normalize=True)
bayesianRidge.fit(train_X, train_y)
regressionSummary(valid_y, bayesianRidge.predict(valid_X))
print('Bayesian ridge chosen regularization: ', bayesianRidge.lambda_ / bayesianRidge.alpha_)


Regression statistics

                      Mean Error (ME) : 120.6311
       Root Mean Squared Error (RMSE) : 1332.2752
            Mean Absolute Error (MAE) : 1021.5286
          Mean Percentage Error (MPE) : -0.2364
Mean Absolute Percentage Error (MAPE) : 9.0115

Regression statistics

                      Mean Error (ME) : 145.1571
       Root Mean Squared Error (RMSE) : 1397.9428
            Mean Absolute Error (MAE) : 1052.4649
          Mean Percentage Error (MPE) : -0.2966
Mean Absolute Percentage Error (MAPE) : 9.2918
Lasso-CV chosen regularization:  3.5138446691310588
[-1.40370575e+02 -1.76669006e-02  3.38674037e+01  0.00000000e+00
  6.94393427e+01  0.00000000e+00  0.00000000e+00  2.70913468e+00
  1.24342596e+01 -0.00000000e+00  0.00000000e+00]

Regression statistics

                      Mean Error (ME) : 154.3286
       Root Mean Squared Error (RMSE) : 1879.7426
            Mean Absolute Error (MAE) : 1353.2735
          Mean Percentage Error (MPE) : -2.3897
Mean Absolu

In [ ]:
### Compare regularization models with normal linear regression 

In [ ]:
linearRegression = LinearRegression(normalize=True).fit(train_X, train_y)
regressionSummary(valid_y, linearRegression.predict(valid_X))


Regression statistics

                      Mean Error (ME) : 103.6803
       Root Mean Squared Error (RMSE) : 1312.8523
            Mean Absolute Error (MAE) : 1017.5972
          Mean Percentage Error (MPE) : -0.2633
Mean Absolute Percentage Error (MAPE) : 9.0111


In [ ]:
pd.DataFrame({'features': train_X.columns, 'linear regression': linearRegression.coef_, 
              'lassoCV': lasso_cv.coef_, 'bayesianRidge': bayesianRidge.coef_})

## Examine the output, note that Lasso model performs slightly worse than normal linear regression
## Lasso to a model with 6 predictors, while the regularization parameter derived with BayesianRidge regression is very small. 
## Overall, this indicates that this dataset does not beneft from regularization
## However, these methods are most beneficial when the dataset contains a large number of highly correlated predictors.

,features,linear regression,lassoCV,bayesianRidge
0,Age_08_04,-140.748761,-140.370575,-139.754059
1,KM,-0.017840,-0.017667,-0.018131
2,HP,36.103419,33.867404,35.856074
3,Met_Color,84.281830,0.000000,85.088966
4,Automatic,416.781954,69.439343,408.599781
5,CC,0.017737,0.000000,0.020405
6,Doors,-50.657863,0.000000,-47.917629
7,Quarterly_Tax,13.625325,2.709135,13.269979
8,Weight,13.038711,12.434260,13.114412
9,Fuel_Type_Diesel,1066.464681,-0.000000,955.581484


## Linear Regression with Statsmodel's sm.ols method (alternative to scikits' lm method)

In [ ]:
# Run a linear regression of Price on the attributes using Statsmodel
train_df = train_X.join(train_y)

predictors = train_X.columns
formula = 'Price ~ ' + ' + '.join(predictors)

car_lm = sm.ols(formula=formula, data=train_df).fit()
print(car_lm.summary())

# compare output ==> similar to lm but more extensive to facilitate non-predictive tasks e.g. statistical inference

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.856
Model:                            OLS   Adj. R-squared:                  0.854
Method:                 Least Squares   F-statistic:                     319.0
Date:                Sat, 06 Feb 2021   Prob (F-statistic):          1.73e-239
Time:                        02:56:09   Log-Likelihood:                -5198.1
No. Observations:                 600   AIC:                         1.042e+04
Df Residuals:                     588   BIC:                         1.047e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept        -1319.3544   1728.427  

In [ ]:
Intercept        -1319.3544   1728.427     -0.763      0.446   -4713.997    2075.288
Age_08_04         -140.7488      5.142    -27.374      0.000    -150.847    -130.650
KM                  -0.0178      0.002     -7.286      0.000      -0.023      -0.013
HP                  36.1034      5.321      6.785      0.000      25.653      46.554
Met_Color           84.2818    127.005      0.664      0.507    -165.158     333.721
Automatic          416.7820    259.794      1.604      0.109     -93.454     927.018
CC                   0.0177      0.099      0.179      0.858      -0.177       0.213
Doors              -50.6579     65.187     -0.777      0.437    -178.686      77.371
Quarterly_Tax       13.6253      2.518      5.411      0.000       8.680      18.571
Weight              13.0387      1.602      8.140      0.000       9.893      16.185
Fuel_Type_Diesel  1066.4647    527.285      2.023      0.044      30.872    2102.057
Fuel_Type_Petrol 

In [ ]:
# For regularization in statsmodel, 
## use OLS.fit_regularized and set argument 
## L1_wt=0 for ridge regression or
## L1_wt=1 for lasso

In [ ]:
# remove p > 0.05 features
newtrain_X = train_X[['Age_08_04', 'KM', 'HP', 'Quarterly_Tax', 'Weight', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol']]
newvalid_X=valid_X[['Age_08_04', 'KM', 'HP', 'Quarterly_Tax', 'Weight', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol']]

In [ ]:
predictors

,Age_08_04,KM,HP,Quarterly_Tax,Weight,Fuel_Type_Diesel,Fuel_Type_Petrol
371,35,18000,110,85,1075,0,1
45,23,84000,90,234,1270,1,0
560,54,39291,110,69,1040,0,1
748,61,81170,110,69,1040,0,1
419,55,94122,86,69,1015,0,1
...,...,...,...,...,...,...,...
767,57,78955,86,69,1045,0,1
72,28,28817,110,85,1130,0,1
908,66,58654,110,85,1070,0,1
235,38,60829,110,85,1075,0,1


In [ ]:
lr = LinearRegression().fit(newtrain_X, train_y)
regressionSummary(valid_y, lr.predict(newvalid_X))


Regression statistics

                      Mean Error (ME) : 93.8702
       Root Mean Squared Error (RMSE) : 1309.9404
            Mean Absolute Error (MAE) : 1015.4177
          Mean Percentage Error (MPE) : -0.3613
Mean Absolute Percentage Error (MAPE) : 8.9971


In [ ]:
lr = LinearRegression().fit(train_X, train_y)
regressionSummary(valid_y, lr.predict(valid_X))


Regression statistics

                      Mean Error (ME) : 103.6803
       Root Mean Squared Error (RMSE) : 1312.8523
            Mean Absolute Error (MAE) : 1017.5972
          Mean Percentage Error (MPE) : -0.2633
Mean Absolute Percentage Error (MAPE) : 9.0111


In [ ]:
# 

         
**Compare with previous models, new model has better lower RMSE(ME\MAE\MPE\MAPE) score for test set. So it is better not use Met_Color Automatic CC Door for the model.**